In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
from preprocessing import inception_preprocessing

In [ ]:
import inception_base

In [ ]:
class Dogs_Cats_model(object):
    def __init__(self,learning_rate=1e-3,num_classes=2,initializer_scale=0.5):     
        self.learning_rate = learning_rate
        self.num_classes = num_classes
        self.initializer = tf.random_uniform_initializer(
                                            minval=-initializer_scale,
                                            maxval=initializer_scale)
        # A float32 Tensor with shape [batch_size, height, width, channels].
        self.images = None

        # A float32 scalar Tensor; the total loss for the trainer to optimize.
        self.final_loss = None

        # Global step Tensor.
        self.global_step = None
    def build_inputs_outputs(self):
        image_feed = tf.placeholder(dtype=tf.float32,shape=[None,299,299,3],name="image_feed")
        self.image_feed = image_feed
        self.ground_truth = tf.placeholder(tf.float32,[None,self.num_classes])
        
    def build_image_embedding(self):
        self.inception_output = inception_base.get_base_model(self.image_feed)
        self.fine_tune_input = tf.placeholder_with_default(self.inception_output,self.inception_output.get_shape(),name="fine_tune_input")
        self.inception_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="InceptionV4")
        with tf.variable_scope("fine_tune_layers") as scope:
            self.logits = tf.contrib.layers.fully_connected(inputs=self.fine_tune_input,
                                                          num_outputs=self.num_classes,
                                                          activation_fn = None,
                                                          weights_initializer=self.initializer,
                                                          biases_initalizer=None,
                                                          scope = scope)
    def build_loss(self):
        self.probabilities = tf.nn.softmax(self.logits,name="output_probs")
        with tf.name_scope("cross_entropy") as scope:
            loss = tf.nn.softmax_cross_entropy_with_logits(labels=self.ground_truth,logits=self.logits)
            with tf.name_scope('total'):
                  self.final_loss = tf.reduce_mean(loss)
    def build_train_step(self):
        with tf.name_scope("train") as scope:
            optimizer = tf.train.AdamOptimizer(self.learning_rate)
            self.train_step = optimizer.minimize(self.final_loss)
            
    def build_eval_step(self):
        with tf.name_scope('accuracy'):
            with tf.name_scope('correct_prediction'):
                prediction = tf.argmax(self.logits, 1)
                correct_prediction = tf.equal(prediction, tf.argmax(self.ground_truth, 1))
            with tf.name_scope('accuracy'):
                self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    def setup_global_step(self):
        """Sets up the global step Tensor."""
        global_step = tf.Variable(
            initial_value=0,
            name="global_step",
            trainable=False,
            collections=[tf.GraphKeys.GLOBAL_STEP, tf.GraphKeys.GLOBAL_VARIABLES])
        self.global_step = global_step
        
    def build(self):
        self.build_inputs_outputs()
        self.build_image_embedding()
        self.build_loss()
        self.build_train_step()
        self.build_eval_step()

## Getting input from file

In [ ]:
# dict containing file_name and label
train_name_dict = {}

In [ ]:
TRAIN_DIR_CAT = "../ignore/kaggle_cats_and_dogs/train/cat/"

In [ ]:
TRAIN_DIR_DOG = "../ignore/kaggle_cats_and_dogs/train/dog/"

In [ ]:
TEST_DIR = "../ignore/kaggle_cats_and_dogs/test1/"

In [ ]:
# list of np.float32 array containing the training images
train_data_unprocessed = []

# list of np.float32 array containing the test images
test_data_unprocessed = []

In [ ]:
cats_files = [os.path.join(TRAIN_DIR_CAT,fl) for fl in os.listdir(TRAIN_DIR_CAT)]
dogs_files = [os.path.join(TRAIN_DIR_DOG,fl) for fl in os.listdir(TRAIN_DIR_DOG)]

In [ ]:
test_files = [os.path.join(TEST_DIR,fl) for fl in os.listdir(TEST_DIR)]

In [ ]:
import scipy

In [ ]:
len(dogs_files)

In [ ]:
import sys

In [ ]:
sys.getsizeof(scipy.ndimage.imread(cats_files[0]))


In [ ]:
cats_images = [scipy.ndimage.imread(fl) for fl in cats_files]

In [ ]:
dogs_images = [scipy.ndimage.imread(fl) for fl in dogs_files]

In [ ]:
#test_images = [scipy.ndimage.imread(fl) for fl in test_files]

## Processing image data

In [ ]:
cats_resized = [scipy.misc.imresize(cat,[299,299,3]) for cat in cats_images]

In [ ]:
dogs_resized = [scipy.misc.imresize(dog,[299,299,3]) for dog in dogs_images]

In [ ]:
#test_resized = [scipy.misc.imresize(tst,[299,299,3]) for tst in test_images]

In [ ]:
cats_processed = [((2*(cat.astype(np.float32) / 255 ))-1) for cat in cats_resized]

In [ ]:
dogs_processed = [((2*(dog.astype(np.float32) / 255 ))-1) for dog in dogs_resized]

In [ ]:
#test_processed = [((2*(tst.astype(np.float32) / 255 ))-1) for tst in test_resized]

In [ ]:
train_data_cat = [(cat,0) for cat in cats_processed]

In [ ]:
train_data_dog = [(dog,1) for dog in dogs_processed]

In [ ]:
train_data_processed=[]

In [ ]:
[train_data_processed.append(cat) for cat in train_data_cat]

In [ ]:
[train_data_processed.append(dog) for dog in train_data_dog]

## Shuffling the array

In [ ]:
import random

In [ ]:
random.shuffle(train_data_processed)

In [ ]:
train_data = [t[0] for t in train_data_processed]

In [ ]:
train_labels = [t[1] for t in train_data_processed]

### Splitting for batchsize

In [ ]:
batch_size = 25

In [ ]:
train_data_np = np.array(train_data)

In [ ]:
train_data_np.shape

## Making model

In [ ]:
model = Dogs_Cats_model()

In [ ]:
model.build()

In [ ]:
sess = tf.Session()

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
saver = tf.train.Saver(var_list=model.inception_variables)
saver.restore(sess,"./inception_v4.ckpt")

## Getting inception output

In [ ]:
sess.run(model.inception_output,feed_dict={model.image_feed=})

In [ ]:
train_inception_output = []

In [ ]:
test_inception_output = []

## Division in batches

In [ ]:
train_inception_batched = []
test_inception_batched = []

## Training layers

In [ ]:
num_epochs = 4

In [ ]:
for _ in xrange(num_epochs):
    for i in xrange(len(train_inception_batched)):
        _,loss = sess.run([model.train_op,model.final_loss],feed_dict={model.fine_tune_input=train_inception_batched[i],model.target})
        print("Loss = ",loss)